In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.16.1


In [11]:
import os
import numpy as np
import cv2
from sklearn.utils import shuffle

In [12]:
# from google.colab import drive
# drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [13]:
from sklearn.model_selection import train_test_split

In [38]:
# # Define paths to your image folders
# non_vehicle_folder = "vehicle_dataset/non-vehicles/"
# vehicle_folder = "vehicle_dataset/vehicles/"

# # Step 1: List Image Files using os.listdir()
# non_vehicle_files = [os.path.join(non_vehicle_folder, file) for file in os.listdir(non_vehicle_folder) if file.endswith('.png')]
# vehicle_files = [os.path.join(vehicle_folder, file) for file in os.listdir(vehicle_folder) if file.endswith('.png')]

# # Step 2: Create Dataset with Labels
# non_vehicle_dataset = tf.data.Dataset.from_tensor_slices((non_vehicle_files, [1] * len(non_vehicle_files)))
# vehicle_dataset = tf.data.Dataset.from_tensor_slices((vehicle_files, [2] * len(vehicle_files)))

# # Step 3: Combine Datasets
# image_dataset = non_vehicle_dataset.concatenate(vehicle_dataset)

images = []
labels = []
dataset = 'vehicle_dataset'

for folder in os.listdir(dataset):
    if not folder.startswith('.'):
        label = None

        if folder in ['vehicles']: label = 0
        elif folder in ['non-vehicles']: label = 1

        for file in os.listdir(os.path.join(dataset, folder)):
            if file.endswith('.png'):
                img_path = os.path.join(os.path.join(dataset, folder), file)

                image = cv2.imread(img_path)

                images.append(image)
                labels.append(label)

images = np.array(images, dtype='float32')/255.0
labels = np.array(labels, dtype='int32')

images = images.reshape(-1,64,64,3)

images, labels = shuffle(images, labels, random_state=42)

# train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size = 0.2)
xtrain,xval,ytrain,yval=train_test_split(images, labels,train_size=0.8,random_state=42)

In [39]:
images[0].shape

(64, 64, 3)

In [46]:
# # Define the model architecture
# class SSDModel(tf.keras.Model):
#     def __init__(self, num_classes):
#         super(SSDModel, self).__init__()
#         self.num_classes = num_classes

#         # Backbone network (e.g. VGG16)
#         self.backbone = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

#         # Feature extractor layers
#         self.feature_extractor = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
#             tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
#             tf.keras.layers.MaxPooling2D((2, 2))
#         ])

#         # SSD layers
#         self.ssd_layers = []
#         for i in range(6):  # 6 default boxes per feature map
#             self.ssd_layers.append(SSDLayer(4 + i, num_classes))

#         # Prediction layers
#         self.prediction_layers = []
#         for i in range(6):  # 6 feature maps
#             self.prediction_layers.append(PredictionLayer(num_classes))

#     def call(self, inputs):
#         # Backbone network
#         x = self.backbone(inputs)

#         # Feature extractor layers
#         x = self.feature_extractor(x)

#         # SSD layers
#         feature_maps = []
#         for i, layer in enumerate(self.ssd_layers):
#             print(layer)
#             x, feature_map = layer(x)
#             feature_maps.append(feature_map)

#         # Prediction layers
#         predictions = []
#         for i, layer in enumerate(self.prediction_layers):
#             predictions.append(layer(feature_maps[i]))

#         # Concatenate predictions
#         predictions = tf.concat(predictions, axis=1)

#         return predictions

# class SSDLayer(tf.keras.layers.Layer):
#     def __init__(self, num_defaults, num_classes):
#         super(SSDLayer, self).__init__()
#         self.num_defaults = num_defaults
#         self.num_classes = num_classes

#     def build(self, input_shape):
#         self.conv = tf.keras.layers.Conv2D(self.num_defaults * (4 + self.num_classes), (3, 3), activation='linear', padding='same', input_shape=input_shape)

#     def call(self, inputs):
#         # inputs = tf.reshape(inputs, (-1, 64, 64, 3))  # Reshape inputs here
#         inputs = tf.reshape(inputs, (-1, 32, 32, 3))  # Assuming 3 channels (RGB)
#         x = self.conv(inputs)
#         x = tf.reshape(x, (-1, x.shape[1], x.shape[2], self.num_defaults, 4 + self.num_classes))
#         return x, x[:, :, :, :, :4]

# class PredictionLayer(tf.keras.layers.Layer):
#     def __init__(self, num_classes):
#         super(PredictionLayer, self).__init__()
#         self.num_classes = num_classes

#     def build(self, input_shape):
#         self.conv = tf.keras.layers.Conv2D(self.num_classes, (3, 3), activation='softmax', padding='same', input_shape=input_shape)

#     def call(self, inputs):
#         x = self.conv(inputs)
#         return x

# def load_image(file_path, label):
#     img = tf.io.read_file(file_path)
#     img = tf.image.decode_image(img, channels=3)  # Ensure 3 channels for color images
#     img = tf.cast(img, tf.uint8)  # Cast the image to uint8 data type
#     return img, label


# # Create the model
# model = SSDModel(num_classes=2)  # 2 classes: vehicle and non-vehicle

# # Compile the model
# model.compile(optimizer='adam', loss='ssd_loss', metrics=['accuracy'])

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(64,64,3)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))
 
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=['accuracy'])

# # Create a dataset object
# dataset = image_dataset.map(load_image)

# # # Define the size of the training set
# # train_size = int(0.8 * dataset.cardinality().numpy())


# # # Split the dataset into training and testing subsets
# # train_dataset = dataset.take(train_size)
# # test_dataset = dataset.skip(train_size)

# # Convert dataset to list to calculate train_size
# dataset_list = list(dataset)
# train_size = int(0.8 * len(dataset_list))

# # Split the dataset into training and testing subsets
# train_dataset = tf.data.Dataset.from_tensor_slices(dataset_list[:train_size])
# test_dataset = tf.data.Dataset.from_tensor_slices(dataset_list[train_size:])



/Users/davidtan/Documents/CS171/FinalProject/Vehicle-Detection-Project/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,    
        rotation_range=15,    
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=False)
datagen.fit(xtrain)

In [48]:
# Train the model
model.fit(xtrain, ytrain, batch_size=32,
                    epochs=5,
                    verbose=1,
                    validation_data=(xval, yval))

Epoch 1/5
444/444 ━━━━━━━━━━━━━━━━━━━━ 97s 215ms/step - accuracy: 0.8663 - loss: 0.3078 - val_accuracy: 0.9724 - val_loss: 0.0739
Epoch 2/5
444/444 ━━━━━━━━━━━━━━━━━━━━ 93s 210ms/step - accuracy: 0.9777 - loss: 0.0709 - val_accuracy: 0.9820 - val_loss: 0.0496
Epoch 3/5
444/444 ━━━━━━━━━━━━━━━━━━━━ 97s 219ms/step - accuracy: 0.9844 - loss: 0.0477 - val_accuracy: 0.9797 - val_loss: 0.0684
Epoch 4/5
444/444 ━━━━━━━━━━━━━━━━━━━━ 96s 215ms/step - accuracy: 0.9851 - loss: 0.0455 - val_accuracy: 0.9870 - val_loss: 0.0353
Epoch 5/5
444/444 ━━━━━━━━━━━━━━━━━━━━ 95s 214ms/step - accuracy: 0.9873 - loss: 0.0402 - val_accuracy: 0.9901 - val_loss: 0.0305
